https://mobiusfunction.wordpress.com/2010/12/08/the-inverse-of-triangular-matrix-as-a-binomial-series/

https://stackoverflow.com/questions/420612/is-there-around-a-straightforward-way-to-invert-a-triangular-upper-or-lower-ma

https://www.google.com/search?q=%22Inverse+of+triangular+matrix%22&safe=active&client=firefox-b-d&ei=OqPlXo7OLtyV5OUPoPGroAI&start=10&sa=N&ved=2ahUKEwiO073vt4DqAhXcCrkGHaD4CiQQ8NMDegQIDRA4&biw=1399&bih=622

Experiment on lower triangular matrix inversion performance

In [ ]:
n = size(A,1);
B = zeros(n);
for i=1:n
    B(i,i) = 1/A(i,i);
    for j=1:i-1
        s = 0;
        for k=j:i-1
            s = s + A(i,k)*B(k,j);
        end
        B(i,j) = -s*B(i,i);
    end
end

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import matrix as mtx, identity as I
from numpy import set_printoptions, tril, triu
from scipy.linalg import solve_triangular


def triangular_inversion(triang_arg):
    # https://dumpz.org/czbtFe4FS8m6
    """Counts inversion of a triangular matrix (lower or upper).

    Args:
        triang_arg (np.matrix, np.array): triangular matrix for inversion

    Returns:
        np.matrix: inverse of triangular matrix
        
    Raises:
        Exception: An error occured while passing non-square matrix
        Exception: An error occured while passing non-triangular matrix
        Exception: An error occured while passing singular matrix

    """
    #if len(triang.shape) != 2 or triang_arg.shape[0] != triang_arg.shape[1]:
    #    raise Exception('Matrix is non-square')
    #if triang_arg not in [tril(triang_arg), triu(triang_arg)]:
    #    raise Exception('Matrix is not triangular')
   # if len(triang_arg.diagonal().nonzero()[0]):
    #    raise Exception('Matrix is singular')

    triang = mtx(triang_arg.copy())
    n = triang.shape[0]

    unitriang_maker = mtx(I(n)) / triang.diagonal()
    unitriang = unitriang_maker * triang
    nilpotent = unitriang - I(n)

    unitriang_inverse = mtx(I(n))
    for i in range(n-1):
        unitriang_inverse = mtx(I(n)) - nilpotent * unitriang_inverse

    return unitriang_inverse * unitriang_maker


In [7]:
N= 20
k = -0.2

L = np.zeros(shape=(N*N, N*N))
i, j = np.indices(L.shape)
L[i==j] = 1 # main diagonal
L[(i==j+1) & ~(np.mod(i, N)==0)] = k # right neighbour
L[(i == j+N) ] = k # lower neighbour

display(L)

array([[ 1. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [-0.2,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. , -0.2,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ..., -0.2,  1. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. , -0.2,  1. ]])

In [8]:
Linv1 = np.linalg.inv(L)
Linv2 = solve_triangular(L, np.eye(*L.shape), lower=True)
Linv3 = triangular_inversion(L)
display(np.allclose(Linv1, Linv2), np.allclose(Linv1, Linv3))

True

True

In [9]:
%timeit -r10 np.linalg.inv(L)
%timeit -r10 solve_triangular(L, np.eye(*L.shape), lower=True)
%timeit -r10 triangular_inversion(L)

9.91 ms ± 2.19 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)
3.85 ms ± 677 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)
2.24 s ± 488 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
